## HTML post-process

`Typora` generates an HTML file with inline CSS styling, with the selected theme at the moment of the export.
The cascade is as follows:

1. Base CSS from Typora, extracted to its own CSS file: `typora.base.css`
1. Selected base theme, in our case the files `monospace-dark.css` or `monospace.css`
1. Custom common CSS for Typora, called `base.user.css` in the Typora themes folder.
1. Custom CSS for each theme, using files with ".user" postfix: `monospace-dark.user.css` or `monospace.user.css`

By using media queries we can implement both themes to be in sync with the viewer local config,
and force any of the 2 themes for `@media print`.

The expected html in `<head>` should contain these CSS:

```html
<link href='typora.base.css' rel='stylesheet' type='text/css' />
<link href='monospace-dark.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference), print"/>
<link href='monospace.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: light)"/>
<link href='base.user.css' rel='stylesheet' type='text/css' />
<link href='monospace-dark.user.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: dark), screen and (prefers-color-scheme: no-preference), print"/>
<link href='monospace.user.css' rel='stylesheet' type='text/css' media="screen and (prefers-color-scheme: light)" />
```

A manual switch (hidden for print) is also added to toggle between themes, by just inserting 2 lines:

```html
<!-- on head -->
<script type="module" src="https://googlechromelabs.github.io/dark-mode-toggle/src/dark-mode-toggle.mjs"></script>
```

```html
<!-- at the end of body -->
<aside>
    <dark-mode-toggle id="dark-mode-toggle-1" appearance="toggle" legend="dark mode" dark="on" light="off" remember="remember"></dark-mode-toggle>
</aside>
```

### Jinja2 template render

As the `head` gets more complex (favicons, open graph meta tags, etc), **it is easier to maintain** it with a jinja2 template, where we fill some basic data for the static site, and insert the content from the extracted `body` in the Typora-generated html.

In [1]:
"""
Script to post-process the Typora-generated web page.

* Replace head with:
 - referenced local CSS files,
 - referenced local badges,
 - Open Graph meta tags,
 - favicon meta tags,
* Add dark/light theme switcher

"""
import re
from pathlib import Path

from jinja2 import Template

# paths
#p_base = Path(__file__).parents[1]
p_base = Path("../")
p_index = p_base / "index.html"
p_typora = p_base / "resumes" / "resume-eugenio-panadero.html"
p_templ = p_base / "notebooks" / "layout.html"
rel_path_rsc = "resources/"

# title and short description
title = "Eugenio Panadero's resume"
description = (
    "Python Senior Developer (12+ years of experience) interested in web apps, "
    "data analysis & ML, IoT, home automation, energy and electrical industries"
)

# Load original html
html_orig = p_typora.read_text()
p_typora.write_text(html_orig.replace("README", title))

body = re.findall(
    r"<body class='typora-export' >(.*)</body>", html_orig, flags=re.DOTALL
)[0]

rg_ref_badge = re.compile(r" src='(https://img.shields.io/badge/(\w+)-.+?)' ")
body_local_badges = rg_ref_badge.sub(r" src='resources/badge_\2.svg' ", body)

# Collect data for template
data = {
    "title": title,
    "short_desc": description,
    "rel_path_rsc": rel_path_rsc,
    "base_url": "https://azogue.github.io",
    "site_name": "profile: Eugenio Panadero",
    "image_width": 868,
    "image_height": 1380,
    "typora_body": body_local_badges,
}

# Render jinja2 template into index.html
p_index.write_text(
    Template(p_templ.read_text()).render(**data)
)

19180